### put keywords merged by Fuzzy matching into the original dataframe

In [2]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
# from fuzzywuzzy.process import dedupe
import functools
from rapidfuzz import process 

In [3]:
# Opening JSON file
f = open('../data/raw/CMS_2010_to_June_2022_ENGLISH.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)

df = pd.DataFrame.from_dict(data)

#print(df.head())

In [4]:
#### Finding the subset of the data for 1 Jan 2019 - 1 Jan 2020 based on lastModifiedDate

In [5]:
df = df.sort_values(by='lastModifiedDate') #sort dataframe

datetimes = pd.to_datetime(df['lastModifiedDate'])
df['ts_lastModifiedDate']=datetimes
#df.iloc[ts_start]['ts_lastModifiedDate']

#find start index for subset 2019-2022
ts_start=datetimes[(datetimes > pd.Timestamp(year=2019, month=1, day=1).tz_localize('utc')) 
          & (datetimes < pd.Timestamp(year=2019, month=1, day=2).tz_localize('utc'))].min()
print(ts_start)
#find end date for subset 2019-2022
ts_end=datetimes[(datetimes > pd.Timestamp(year=2022, month=1, day=1).tz_localize('utc')) 
          & (datetimes < pd.Timestamp(year=2022, month=1, day=2).tz_localize('utc'))].min()
print(ts_end)

start_date=datetimes[datetimes == ts_start]
end_date=datetimes[datetimes == ts_end]

#find index for the chosen start and end dates
start_index=start_date.index[0]
print(start_index)
df[df.index == start_date.index[0]]

end_index=end_date.index[0]
print(end_index)
df[df.index == end_date.index[0]]

df_subset=df[start_index:end_index]


2019-01-01 03:57:28.904000+00:00
2022-01-01 02:35:51.098000+00:00
60278
150367


## put output of fuzzyFuzzy into the dataframe

In [6]:
#8000 keywords from the original 10k
file='../data/interim/out_dedupl_10k_kw_only_140323_threshold_90.csv'
# # #read from csv output of FuzzyWuzzy DEDUP
ded_kw=pd.read_csv(file)

In [7]:
ded_kw

,Unnamed: 0,0
0,0,malware
1,1,state repression
2,2,climate consensus
3,3,Canary Islands
4,4,Angela I
...,...,...
8330,8330,Albertville
8331,8331,Asmaa Abdalla
8332,8332,factory farming
8333,8333,Deichtorhallen Hamburg


In [14]:
deduplicated=list(ded_kw['0'])

In [9]:
dedupl

['malware',
 'state repression',
 'climate consensus',
 'Canary Islands',
 'Angela I',
 'Evan Rachel Wood',
 'used cars',
 'woodcarving',
 'taboo',
 'Grüne Woche',
 'Dugong',
 'Ghassem Soleimani',
 'Food Truck',
 'Israel lockdown',
 'Philippe Monguillot',
 'Boko Haram',
 'US Capitol violence',
 'Benedikt Taschen',
 'Charlemagne Prize',
 'Yahya Jammeh',
 'Middle Ages',
 'InternetLab',
 'Host',
 'Mosambique',
 'child vaccinations',
 'Abel prize',
 'Thierry Breton',
 'work-life balance',
 'Mossad',
 'digital contact tracing',
 'Air Traffic Control',
 'Omar Farouq',
 'Tyrannosaurus rex',
 'Israeli Supreme Court',
 'Jean Paul Gaultier',
 'steel industry',
 'Mikhail Gorbatchev',
 'Krzyzowa',
 'Domestic violence',
 'Jovenel Moise',
 'Global Warmining',
 'forest management',
 'Vatican Apostolic Archive',
 'myoelectric',
 'Italian Democratic Party',
 'Dominican Republic',
 'Verbot',
 'Robert Zoellick',
 'President Abdelaziz Bouteflika',
 'Fight',
 'banknotes',
 'De Telegraaf',
 'Pipe Shelf',
 '

In [55]:
%%time
## speeding up put_clean_kw_into_df
# 1) apply to every over each line in dataframe
# 2) loop over each keyword in the line
# 3) find process.extractOne a substitute from deduplicated list
# 4) create a new column in dataframe with merged keywords


def put_clean_kw_into_df_fast(row: list) -> list:
    #df.applymap(lambda x: len(str(x)))
    line_wr = []
    #ratio_line_wr = []

    for n in range(0, len(row)):
        #print(process.extractOne(df_subset['keywordStrings'][i][n],deduplicated)) #print word and ratio 
        line_wr.append(process.extractOne(row[n], deduplicated)[0])
        #ratio_line_wr.append(process.extractOne(row[n], deduplicated)[1])

    return line_wr


df_subset['keywordStrings'].iloc.apply(put_clean_kw_into_df_fast)


CPU times: user 1min 5s, sys: 178 ms, total: 1min 5s
Wall time: 1min 25s


60278      [NSA, OSIRIS-REx, Benny Gantz, double asteroid]
60279    [English Channel crossings, Remigration, ghost...
60280    [Brazil military, Jair Bolsanaro, Economics, A...
60281    [emperor of Japan, Paralympics Tokyo 2020, Kar...
60282    [Radio Free Asia, Bangaldesh, elections, Al Ho...
                               ...                        
61273    [red meat consumption, "yellow vests", Air Fra...
61274             [Heiscreen, Transas, African filmmakers]
61275    [Emmanuelle Macron, Abdel Fatah el-Sissi, anci...
61276    [women's participation, RAF, euromaxx, EU comp...
61277                                  [speakup barometer]
Name: keywordStrings, Length: 1000, dtype: object

In [ ]:
#df_lines=put_clean_kw_into_df(df_subset['keywordStrings'])

In [20]:
type(df_lines)

list

In [16]:
df_lines

[['NSA', 'OSIRIS-REx', 'Benny Gantz', 'double asteroid'],
 ['English Channel crossings',
  'Remigration',
  'ghost boats',
  'illegal immigration'],
 ['Brazil military', 'Jair Bolsanaro', 'Economics', 'AOC', 'Goncalo Guedes'],
 ['emperor of Japan', 'Paralympics Tokyo 2020', 'Karaj', 'bomb attack'],
 ['Radio Free Asia',
  'Bangaldesh',
  'elections',
  'Al Hol',
  'Sheikh Hasina',
  'Major League Baseball',
  'IAA'],
 ['Iranian nuclear program',
  'UN sanctions',
  'Syrian American Medical Society',
  'Iran nuclear deal',
  'JCPoA',
  'UN sanctions',
  'Tehran',
  'Rouhani',
  'Ayatollah Khamenei']]